# Data Preparation
Before analyzing Apple’s emissions data, it is important to ensure that all sources are clean, consistent, and easy to join. The raw CSVs from sustainability reports contain mixed formats, varying column names, and data spread across multiple files. Using SQLite allows me to store all datasets in one place, standardize column names and types, filter to the target years (2015–2022), and create views that combine key metrics. This preparation step ensures the data is reliable and ready for deeper analysis in Python.

In [16]:
import pandas as pd
import sqlite3

In [17]:
#creating database connection
conn = sqlite3.connect('emission.db')

In [21]:
#creating tables present in the database

# See any tables that look like checkpoints
pd.read_sql_query("""
SELECT name FROM sqlite_master 
WHERE type='table' AND name LIKE '.ipynb_check%';
""", conn)

# Drop them
for t in pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' AND name LIKE '.ipynb_check%';", 
    conn
)['name']:
    conn.execute(f'DROP TABLE IF EXISTS "{t}";')

conn.commit()

# Verify
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)

,name
0,carbon_footprint_by_product
1,data_dictionary
2,greenhouse_gas_emissions
3,normalizing_factors


In [20]:
# Explore the schema in SQL

for t in ["greenhouse_gas_emissions","carbon_footprint_by_product","normalizing_factors"]:
    print(f"--- {t} ---")
    display(pd.read_sql_query(f"PRAGMA table_info({t});", conn))


--- greenhouse_gas_emissions ---


,cid,name,type,notnull,dflt_value,pk
0,0,Fiscal Year,BIGINT,0,None,0
1,1,Category,TEXT,0,None,0
2,2,Type,TEXT,0,None,0
3,3,Scope,TEXT,0,None,0
4,4,Description,TEXT,0,None,0
5,5,Emissions,FLOAT,0,None,0


--- carbon_footprint_by_product ---


,cid,name,type,notnull,dflt_value,pk
0,0,Release Year,BIGINT,0,None,0
1,1,Product,TEXT,0,None,0
2,2,Baseline Storage,BIGINT,0,None,0
3,3,Carbon Footprint,BIGINT,0,None,0


--- normalizing_factors ---


,cid,name,type,notnull,dflt_value,pk
0,0,Fiscal Year,BIGINT,0,None,0
1,1,Revenue,BIGINT,0,None,0
2,2,Market Capitalization,BIGINT,0,None,0
3,3,Employees,BIGINT,0,None,0


# Data Cleaning
The raw Apple sustainability CSVs contain inconsistent column names, data types, and formats. For example, numeric values may be stored as text or include commas, and year columns are labeled differently across files (Fiscal Year, Release Year).




In [56]:
# Check for null values in important columns
for view in ["greenhouse_gas_emissions","carbon_footprint_by_product","normalizing_factors"]:
    print(f"\n--- NULL check for {view} ---")
    df = pd.read_sql_query(f"SELECT * FROM {view};", conn)
    print(df.isnull().sum())



--- NULL check for greenhouse_gas_emissions ---
Fiscal Year     0
Category        0
Type            0
Scope          16
Description     0
Emissions      27
dtype: int64

--- NULL check for carbon_footprint_by_product ---
Release Year        0
Product             0
Baseline Storage    0
Carbon Footprint    0
dtype: int64

--- NULL check for normalizing_factors ---
Fiscal Year              0
Revenue                  0
Market Capitalization    0
Employees                0
dtype: int64


In [58]:
pd.read_sql_query("""
SELECT *
FROM greenhouse_gas_emissions
WHERE scope IS NULL;
""", conn)

,Fiscal Year,Category,Type,Scope,Description,Emissions
0,2022,Corporate emissions,Carbon removals,None,Corporate carbon offsets,-324100.0
1,2022,Product life cycle emissions,Carbon removals,None,Product carbon offsets,0.0
2,2021,Corporate emissions,Carbon removals,None,Corporate carbon offsets,-167000.0
3,2021,Product life cycle emissions,Carbon removals,None,Product carbon offsets,-500000.0
4,2020,Corporate emissions,Carbon removals,None,Corporate carbon offsets,-70000.0
5,2020,Product life cycle emissions,Carbon removals,None,Product carbon offsets,0.0
6,2019,Corporate emissions,Carbon removals,None,Corporate carbon offsets,0.0
7,2019,Product life cycle emissions,Carbon removals,None,Product carbon offsets,0.0
8,2018,Corporate emissions,Carbon removals,None,Corporate carbon offsets,0.0
9,2018,Product life cycle emissions,Carbon removals,None,Product carbon offsets,0.0


Carbon removals have no scope in the raw data because they are outside Scope 1, 2, and 3. I will label them as “Outside scopes (removals)” so they are easier to track in charts without mixing them into scope totals. I am also renaming columns like emissions_tonnes and carbon_footprint_kg to make the units clear.

In [67]:
pd.read_sql_query("""
SELECT *
FROM greenhouse_gas_emissions
WHERE emissions IS NULL;
""", conn)

,Fiscal Year,Category,Type,Scope,Description,Emissions
0,2021,Corporate emissions,Gross emissions,Scope 3,Transmission and distribution loss (market-based),None
1,2020,Corporate emissions,Gross emissions,Scope 3,Transmission and distribution loss (market-based),None
2,2019,Corporate emissions,Gross emissions,Scope 3,Transmission and distribution loss (market-based),None
3,2018,Corporate emissions,Gross emissions,Scope 3,Transmission and distribution loss (market-based),None
4,2017,Corporate emissions,Gross emissions,Scope 2 (market-based),"Steam, heating, and cooling",None
5,2017,Corporate emissions,Gross emissions,Scope 3,Upstream fuel,None
6,2017,Corporate emissions,Gross emissions,Scope 3,Work from home (market-based),None
7,2017,Corporate emissions,Gross emissions,Scope 3,Transmission and distribution loss (market-based),None
8,2017,Corporate emissions,Gross emissions,Scope 3,Third-party cloud (market-based),None
9,2017,Corporate emissions,Carbon removals,None,Corporate carbon offsets,None


Some rows in the source data have no emission values, usually for minor categories or early carbon removal projects. I am leaving these as missing instead of filling them in, because Apple did not report a number and guessing would reduce accuracy. These NULLs will not affect totals since SQL ignores them when summing.

In [85]:
# Create cleaned views with simpler column names (2015–2022)
conn.executescript("""
DROP VIEW IF EXISTS emissions;
DROP VIEW IF EXISTS products;
DROP VIEW IF EXISTS normalizing_factors_view;

-- Emissions (metric tonnes CO2e)a
CREATE VIEW emissions AS
SELECT
  CAST("Fiscal Year" AS INT) AS year,
  "Category"    AS category,
  "Type"        AS type,
  COALESCE(
    "Scope",
    CASE WHEN "Type" = 'Carbon removals' THEN 'Outside scopes (removals)' END
  ) AS scope,
  "Description" AS description,
  CAST("Emissions" AS REAL) AS emissions_tonnes
FROM greenhouse_gas_emissions
WHERE CAST("Fiscal Year" AS INT) BETWEEN 2015 AND 2022;

-- Products (kg CO2e per device)
CREATE VIEW products AS
SELECT
  CAST("Release Year" AS INT) AS year,
  "Product"           AS product,
  CAST("Baseline Storage" AS INT) AS baseline_storage,
  CAST("Carbon Footprint" AS REAL) AS carbon_footprint_kg
FROM carbon_footprint_by_product
WHERE CAST("Release Year" AS INT) BETWEEN 2015 AND 2022;

-- Normalizing factors
CREATE VIEW normalizing_factors_view AS
SELECT
  CAST("Fiscal Year" AS INT) AS year,
  CAST("Revenue" AS REAL)                AS revenue_usd_b,
  CAST("Market Capitalization" AS REAL)  AS market_cap_usd_b,
  CAST("Employees" AS REAL)              AS employees
FROM normalizing_factors
WHERE CAST("Fiscal Year" AS INT) BETWEEN 2015 AND 2022;
""")
conn.commit()


I created SQL views (emissions, products, and normalizing_factors_view) to clean, standardize, and filter the raw data before analysis. This ensured consistent column naming, correct data types, and a focus on the 2015–2022 period. Using views keeps the raw data intact, makes joins and aggregations easier, and allows all cleaning logic to be maintained in one place for reproducibility.

In [103]:
pd.read_sql_query("SELECT name, type FROM sqlite_master WHERE type='view';", conn)


,name,type
0,emissions,view
1,products,view
2,normalizing_factors_view,view
3,iphone_baseline_by_year,view
4,yearly_summary,view


In [87]:
pd.read_sql_query("SELECT * FROM emissions LIMIT 5;", conn)


,year,category,type,scope,description,emissions_tonnes
0,2022,Corporate emissions,Gross emissions,Scope 1,"Natural gas, diesel, propane",39700.0
1,2022,Corporate emissions,Gross emissions,Scope 1,Fleet vehicles,12600.0
2,2022,Corporate emissions,Gross emissions,Scope 1,Other (R&D processes & refrigerant leaks),2900.0
3,2022,Corporate emissions,Gross emissions,Scope 2 (market-based),Electricity,0.0
4,2022,Corporate emissions,Gross emissions,Scope 2 (market-based),"Steam, heating, and cooling",3000.0


In [88]:
pd.read_sql_query("SELECT * FROM products LIMIT 5;", conn)


,year,product,baseline_storage,carbon_footprint_kg
0,2022,iPhone 14,128,61.0
1,2021,iPhone 13,128,64.0
2,2020,iPhone 12,64,70.0
3,2019,iPhone 11,64,72.0
4,2018,iPhone Xs,64,70.0


In [89]:
pd.read_sql_query("SELECT * FROM normalizing_factors_view LIMIT 5;", conn)

,year,revenue_usd_b,market_cap_usd_b,employees
0,2022,394328.0,2490.0,164000.0
1,2021,365817.0,2450.0,154000.0
2,2020,274515.0,1720.0,147000.0
3,2019,260174.0,1090.0,137000.0
4,2018,265595.0,830.0,132000.0


In [90]:
# Build the iPhone baseline per year

conn.executescript("""
DROP VIEW IF EXISTS iphone_baseline_by_year;

-- Choose the smallest storage per year
CREATE VIEW iphone_baseline_by_year AS
WITH ranked AS (
  SELECT
    year, product, baseline_storage, carbon_footprint_kg,
    ROW_NUMBER() OVER (PARTITION BY year ORDER BY baseline_storage ASC) AS rn
  FROM products
  WHERE product LIKE '%iPhone%'
)
SELECT year, carbon_footprint_kg AS iphone_baseline_kg
FROM ranked
WHERE rn = 1
ORDER BY year;
""")
conn.commit()

pd.read_sql_query("SELECT * FROM iphone_baseline_by_year;", conn)


,year,iphone_baseline_kg
0,2015,54.0
1,2016,56.0
2,2017,79.0
3,2018,70.0
4,2019,72.0
5,2020,70.0
6,2021,64.0
7,2022,61.0


The iPhone is Apple’s flagship product and a major source of product life cycle emissions. To track sustainability at the product level, we use the “baseline” model (smallest storage each year) as a consistent reference. For example, the baseline iPhone’s footprint was 54 kg CO₂e in 2015, rose to 79 kg in 2017, and dropped to 61 kg in 2022. This trend shows how per-device emissions have changed over time and lets us compare product-level progress with Apple’s overall emissions reductions.

In [91]:
# Total carbon emission in tonns by year
pd.read_sql_query("""
SELECT year, ROUND(SUM(emissions_tonnes),2) AS total_mtco2e
FROM emissions
GROUP BY year
ORDER BY year;
""", conn)


,year,total_mtco2e
0,2015,38383470.0
1,2016,29579370.0
2,2017,27416300.0
3,2018,25136170.0
4,2019,25033730.0
5,2020,22524440.0
6,2021,22519400.0
7,2022,20279900.0


In [92]:
pd.read_sql_query("""
SELECT year, scope, ROUND(SUM(emissions_tonnes),2) AS mtco2e
FROM emissions
GROUP BY year, scope
ORDER BY year, scope;
""", conn)


,year,scope,mtco2e
0,2015,Outside scopes (removals),NaN
1,2015,Scope 1,28100.0
2,2015,Scope 2 (market-based),42460.0
3,2015,Scope 3,38312910.0
4,2016,Outside scopes (removals),NaN
5,2016,Scope 1,34370.0
6,2016,Scope 2 (market-based),41000.0
7,2016,Scope 3,29504000.0
8,2017,Outside scopes (removals),NaN
9,2017,Scope 1,47050.0


Apple’s total greenhouse gas emissions dropped from 38.38 million tonnes CO₂e in 2015 to 20.28 million tonnes in 2022 — a 47% reduction. The majority of emissions come from Scope 3 activities (supply chain and product life cycle), which fell from 38.31M tonnes to 20.55M tonnes over the same period. Scope 1 and Scope 2 emissions are comparatively small, while carbon removals, now labeled as “Outside scopes (removals)”, appear as negative or zero values in recent years to reflect offset projects. Apple’s flagship product, the iPhone, remains a major Scope 3 contributor, making it important to track whether reductions in its baseline carbon footprint align with overall corporate emissions progress.

In [100]:

# Create the merged yearly summary
conn.executescript("""
DROP VIEW IF EXISTS yearly_summary;

CREATE VIEW yearly_summary AS
WITH gross AS (
    SELECT year, SUM(emissions_tonnes) AS total_gross_tonnes
    FROM emissions
    WHERE type <> 'Carbon removals'
    GROUP BY year
),
rem AS (
    SELECT year, SUM(emissions_tonnes) AS total_removals_tonnes
    FROM emissions
    WHERE type = 'Carbon removals'
    GROUP BY year
)
SELECT
    g.year,
    g.total_gross_tonnes,
    r.total_removals_tonnes,
    (g.total_gross_tonnes + COALESCE(r.total_removals_tonnes, 0)) AS total_net_tonnes,
    nf.revenue_usd_b,
    nf.market_cap_usd_b,
    nf.employees,
    ib.iphone_baseline_kg
FROM gross g
LEFT JOIN rem r ON r.year = g.year
LEFT JOIN normalizing_factors_view nf ON nf.year = g.year
LEFT JOIN iphone_baseline_by_year ib ON ib.year = g.year
ORDER BY g.year;
""")
conn.commit()


The yearly_summary view combines the key metrics needed for high-level analysis into one table. From the cleaned emissions data (emissions_clean), it separately calculates gross emissions (all sources except removals), carbon removals, and net emissions (gross + removals) for each year. It then joins this with the normalizing factors (revenue, market capitalization, employees) and the baseline iPhone carbon footprint. These columns give a clear picture of overall emissions trends, the role of removals in net reductions, efficiency relative to business growth, and product-level impacts.

Before analysis, SQL steps were used to:

- Standardize column names and convert values to correct types (CAST and COALESCE).

- Label missing scope values for carbon removals as "Outside scopes (removals)" without imputing missing emission amounts.

- Filter all records to the target analysis period (2015–2022).

- Create intermediate cleaned views (emissions, emissions_clean, products, iphone_baseline_by_year, normalizing_factors_view).

- Merge them into the yearly_summary view with clear gross, removals, and net columns.

This process, implemented via conn.executescript(...), ensures the dataset is complete, consistent, and ready for reliable trend analysis, emissions intensity calculations, and 2030 net-zero progress forecasting.

In [102]:
# Summary rows (should be 2015–2022)
pd.read_sql_query("SELECT * FROM yearly_summary;", conn)

,year,total_gross_tonnes,total_removals_tonnes,total_net_tonnes,revenue_usd_b,market_cap_usd_b,employees,iphone_baseline_kg
0,2015,38383470.0,NaN,38383470.0,233715.0,580.0,110000.0,54.0
1,2016,29579370.0,NaN,29579370.0,215639.0,600.0,116000.0,56.0
2,2017,27416300.0,NaN,27416300.0,229234.0,740.0,123000.0,79.0
3,2018,25136170.0,0.0,25136170.0,265595.0,830.0,132000.0,70.0
4,2019,25033730.0,0.0,25033730.0,260174.0,1090.0,137000.0,72.0
5,2020,22594440.0,-70000.0,22524440.0,274515.0,1720.0,147000.0,70.0
6,2021,23186400.0,-667000.0,22519400.0,365817.0,2450.0,154000.0,64.0
7,2022,20604000.0,-324100.0,20279900.0,394328.0,2490.0,164000.0,61.0


In [101]:
 # Quick consistency check: net = gross + removals
pd.read_sql_query("""
SELECT year,
       total_gross_tonnes,
       total_removals_tonnes,
       total_net_tonnes,
       (total_gross_tonnes + COALESCE(total_removals_tonnes,0)) AS recomputed_net
FROM yearly_summary;
""", conn)


,year,total_gross_tonnes,total_removals_tonnes,total_net_tonnes,recomputed_net
0,2015,38383470.0,NaN,38383470.0,38383470.0
1,2016,29579370.0,NaN,29579370.0,29579370.0
2,2017,27416300.0,NaN,27416300.0,27416300.0
3,2018,25136170.0,0.0,25136170.0,25136170.0
4,2019,25033730.0,0.0,25033730.0,25033730.0
5,2020,22594440.0,-70000.0,22524440.0,22524440.0
6,2021,23186400.0,-667000.0,22519400.0,22519400.0
7,2022,20604000.0,-324100.0,20279900.0,20279900.0
